<a href="https://colab.research.google.com/github/chrismoroney/natural-language-processing/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!gdown --id 10Yv6xUd1ufDilPcLdhR8L-zQZX5yNwSZ

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10Yv6xUd1ufDilPcLdhR8L-zQZX5yNwSZ
To: /content/dem-vs-rep.zip
100% 9.29M/9.29M [00:00<00:00, 29.4MB/s]


In [ ]:
import os
import zipfile

local_zip = './dem-vs-rep.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [ ]:
train_tweets = os.path.join('./dem-vs-rep/train.csv')
test_tweets = os.path.join('./dem-vs-rep/test.csv')

In [ ]:
import pandas
train_df = pandas.read_csv(train_tweets)
test_df = pandas.read_csv(test_tweets)
print(train_df[:1])
print(test_df[:1])

      Party         Handle                                              Tweet
0  Democrat  RepDarrenSoto  Today, Senate Dems vote to #SaveTheInternet. P...
      Party        Handle                                              Tweet
0  Democrat  RepAdamSmith  Today the House passed an omnibus spending bil...


In [ ]:
import string

stopwords = ["a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at",
             "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do",
             "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having",
             "he", "hed", "hes", "her", "here", "heres", "hers", "herself", "him", "himself", "his", "how",
             "hows", "i", "id", "ill", "im", "ive", "if", "in", "into", "is", "it", "its", "itself",
             "lets", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought",
             "our", "ours", "ourselves", "out", "over", "own", "same", "she", "shed", "shell", "shes", "should",
             "so", "some", "such", "than", "that", "thats", "the", "their", "theirs", "them", "themselves", "then",
             "there", "theres", "these", "they", "theyd", "theyll", "theyre", "theyve", "this", "those", "through",
             "to", "too", "under", "until", "up", "very", "was", "we", "wed", "well", "were", "weve", "were",
             "what", "whats", "when", "whens", "where", "wheres", "which", "while", "who", "whos", "whom", "why",
             "whys", "with", "would", "you", "youd", "youll", "youre", "youve", "your", "yours", "yourself",
             "yourselves"]

table = str.maketrans('', '', string.punctuation)

In [ ]:
import numpy as np

train_tweets = []
train_labels = []

for i in range(len(train_df.index)):
  pre_tweet = train_df.iloc[i]['Tweet']
  post_tweet = ""
  for word in pre_tweet.split():
    if word not in stopwords:
      post_tweet += word + " "
  train_tweets.append(post_tweet)
  party = (train_df.iloc[i]['Party'])
  if party == 'Democrat':
    train_labels.append(0)
  else:
    train_labels.append(1)

test_tweets = []
test_labels = []

for i in range(len(test_df.index)):
  pre_tweet = train_df.iloc[i]['Tweet']
  post_tweet = ""
  for word in pre_tweet.split():
    if word not in stopwords:
      post_tweet += word + " "
  test_tweets.append(post_tweet)
  party = (test_df.iloc[i]['Party'])
  if party == 'Democrat':
    test_labels.append(0)
  else:
    test_labels.append(1)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [ ]:
print(len(train_tweets))

69107


In [ ]:
print(train_tweets[:2])
print(train_labels[:2])

print(test_tweets[:2])
print(test_labels[:2])

['Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L ', 'RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher one several recognized @RepDarrenSoto National Teacher Apprecia… ']
[0 0]
['Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L ', 'RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher one several recognized @RepDarrenSoto National Teacher Apprecia… ']
[0 0]


In [ ]:
vocab_size = 69701
embedding_dim = 16
max_length = 200
padding_type='post'
trunc_type='post'
OOV_token = "<OOV>"

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=OOV_token)
tokenizer.fit_on_texts(train_tweets)

word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_tweets)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(test_tweets)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [ ]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(train_padded[0]))
print(train_tweets[0])

today senate dems vote savetheinternet proud support similar netneutrality legislation house… https t co n3tggdlu1l ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Today, Senate Dems vote #SaveTheInternet. Proud support similar #NetNeutrality legislation House… https://t.co/n3tggDLU1L 


In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 16)           1115216   
                                                                 
 flatten (Flatten)           (None, 3200)              0         
                                                                 
 dense (Dense)               (None, 6)                 19206     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 1,134,429
Trainable params: 1,134,429
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 20
model.fit(train_padded, train_labels, epochs=num_epochs,
          validation_data=(testing_padded, test_labels))

Epoch 1/20
2160/2160 [==============================] - 39s 18ms/step - loss: 0.2327 - accuracy: 0.9194 - val_loss: 0.2940 - val_accuracy: 0.9437
Epoch 2/20
2160/2160 [==============================] - 42s 20ms/step - loss: 0.1087 - accuracy: 0.9690 - val_loss: 0.3567 - val_accuracy: 0.9465
Epoch 3/20
2160/2160 [==============================] - 43s 20ms/step - loss: 0.0712 - accuracy: 0.9805 - val_loss: 0.4885 - val_accuracy: 0.9527
Epoch 4/20
2160/2160 [==============================] - 41s 19ms/step - loss: 0.0446 - accuracy: 0.9872 - val_loss: 0.5619 - val_accuracy: 0.9520
Epoch 5/20
2160/2160 [==============================] - 38s 18ms/step - loss: 0.0253 - accuracy: 0.9926 - val_loss: 0.6255 - val_accuracy: 0.9528
Epoch 6/20
2160/2160 [==============================] - 42s 20ms/step - loss: 0.0145 - accuracy: 0.9964 - val_loss: 0.8059 - val_accuracy: 0.9530
Epoch 7/20
2160/2160 [==============================] - 42s 19ms/step - loss: 0.0102 - accuracy: 0.9978 - val_loss: 0.7018 -

In [ ]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(69701, 16)


<h1> Input statement below </h1>
<p> Insert a sentence down below, then run the remaining cells to clasify your statement </p>


In [ ]:
sentence = ["There are too many shootings in the US. We need change."]

<p> Run the remaining cells to determine the model's interpretation of a democratic or repulican statement <p>

In [ ]:
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[250, 923, 1286, 87, 2470, 32, 8, 24, 13, 58, 520]]


In [ ]:
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print("Result:", model.predict(padded)[0][0])
print()
result = ""
if model.predict(padded)[0][0] < 0.37:
  result = "Republican"
elif model.predict(padded)[0][0] > 0.63:
  result = "Democratic"
else:
  result = "Neutral"
print("Output:", result)

1/1 [==============================] - 0s 26ms/step
Result: 0.003382092

1/1 [==============================] - 0s 22ms/step
Output: Republican


<h1> Interpretting the results <h1>
<p> Look at the value in your cell. The value is a number between 0 and 1.


*   If your number is closer to 0, your statement aligns closer to a ***Republican*** statement
*   If your number is closer to 1, your statement aligns closer to a ***Democratic*** statement